In [1]:
import time, gc
import tensorflow as tf
import numpy as np

import directories, experiments, data_methods, network_methods, metrics

user_name = "jamin"
append_to_tuning_txt = True


In [2]:
EXPERIMENTS = ["tune_code1",]
for EXP in EXPERIMENTS:

    base_dirs = directories.get_dirs()
    start_time = time.localtime()

    print('*** Preparing data for ' + EXP)

    # I.D. experiment settings
    settings = experiments.get_settings(EXP)
    print(settings)

    Atr, Ava, Ate, Aob, Ftr, Fva, Fte, Itr, Iva, Ite, \
            Xtr, Xva, Xte, Xob, Xshapes, Ttr, Tva, Tte, Tshapes, \
            Tmean, Tstd, lats, lons, M = data_methods.prepare_data(settings, base_dirs)


    # Train the model
    print('*** Training model for ' + EXP)
    start_train_time = time.time()
    tf.keras.utils.set_random_seed(settings['seed'])
    model, encoder, decoder = network_methods.train_CVED(Xtr, Ttr, Xva, Tva, settings,verbose=0)
    print('Time elapsed for training: ', time.time() - start_train_time)

    # Make predictions
    print('*** Making predictions for ' + EXP)
    Ptr = model.predict(Xtr)
    _ = gc.collect()
    Pva = model.predict(Xva)
    _ = gc.collect()
    Pte = model.predict(Xte)
    _ = gc.collect()
    PItr = data_methods.unstandardize_predictions(Ptr, Tmean, Tstd, Tshapes[0], M)
    PIva = data_methods.unstandardize_predictions(Pva, Tmean, Tstd, Tshapes[1], M)
    PIte = data_methods.unstandardize_predictions(Pte, Tmean, Tstd, Tshapes[2], M)
    PFtr = Atr[..., 0:1] - PItr
    PFva = Ava[..., 0:1] - PIva
    PFte = Ate[..., 0:1] - PIte

    from multiprocessing import Pool

    # Compute R2 score
    test_IV_R2_str = 'test R2 of IV: %.3f' % metrics.R2(Ite, PIte, weighted=True, lats=lats)
    print(test_IV_R2_str)

    val_IV_R2_str = 'val R2 of IV: %.3f' % metrics.R2(Iva, PIva, weighted=True, lats=lats)
    print(val_IV_R2_str)

    # Compute MAE
    test_IV_MAE_str = 'test MAE of IV: %.3f' % metrics.MAE(Ite, PIte, weighted=True, lats=lats)
    print(test_IV_MAE_str)

    val_IV_MAE_str = 'val MAE of IV: %.3f' % metrics.MAE(Iva, PIva, weighted=True, lats=lats)
    print(val_IV_MAE_str)

    #Compute MAE of global-mean
    test_IV_gMAE_str = 'test gMAE of IV: %.3f' % metrics.globalMAE(Ite, PIte, lats)
    print(test_IV_gMAE_str)

    val_IV_gMAE_str = 'val gMAE of IV: %.3f' % metrics.globalMAE(Iva, PIva, lats)
    print(val_IV_gMAE_str)

    # Compute mean error of global-mean
    test_IV_gE_str = 'test gE of IV: %.3f' % metrics.globalE(Ite, PIte, lats)
    print(test_IV_gE_str)

    val_IV_gE_str = 'val gE of IV: %.3f' % metrics.globalE(Iva, PIva, lats)
    print(val_IV_gE_str)

    # # Compute MAESS of 15-yr trends
    # slope_len=15
    # with Pool(6) as p:
    #     all_slopes_list = p.map(data_methods.calc_slopes(slope_len=slope_len), [Fte, PFte, Ate])

    # SFte, SPFte, SAte = all_slopes_list

    # SFte = np.array(SFte)
    # SPFte = np.array(SPFte)
    # SAte = np.array(SAte)

    # # Calc MAESS
    # skill_score = (1 - np.abs(np.array(SPFte) - np.array(SFte)).mean(axis=(0,1,2,3,5,6)) / np.abs(np.array(SAte) - np.array(SFte)).mean(axis=(0,1,2,3,5,6)))
    # maess_trend_str = str(slope_len) + '-yr Trends MAESS: %.3f' % metrics.weighted_mean(skill_score, lats, lat_axis=0, axis=None)
    # print(maess_trend_str)

    if append_to_tuning_txt:
        with open("tuning_results_" + user_name + ".txt", "a") as myfile:
            myfile.write(EXP + \
                         "\n" + test_IV_R2_str + '\n' + test_IV_MAE_str + \
                         "\n" + test_IV_gMAE_str + '\n' + test_IV_gE_str + \
                         "\n" + val_IV_R2_str + '\n' + val_IV_MAE_str + \
                         "\n" + val_IV_gMAE_str + '\n' + val_IV_gE_str + '\n' + '\n' )

*** Preparing data for main3_noboot
{'data_fn': 'standard_tos_annual.npz', 'obs_fn': 'era5_sst_annual_regrid.nc', 'split_members': ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]), 'data_aug_method': None, 'alter_forced_method': None, 'in_stand_method': 'self_by_member_mean_only', 'out_stand_method': 'gridpoint_stand', 'seed': 0, 'learn_rate': 0.001, 'patience': 15, 'batch_size': 64, 'max_epochs': 5, 'encoding_nodes': [100, 100], 'code_nodes': 100, 'activation': 'tanh', 'ridge': 0.0, 'variational': False, 'variational_loss': 0.0001, 'bias_init': 'random', 'kernel_init': 'random', 'skip': True, 'conv_blocks': ([['Skip'], ['Conv', 32, 3, 1, 'relu'], ['Conv', 32, 3, 1, 'relu'], ['MaxP', 2], ['Skip']], [['Conv', 32, 3, 1, 'relu'], ['Conv', 32, 3, 1, 'relu'], ['MaxP', 2], ['Skip']]), 'split_models': ([0, 1, 2, 4], [0, 1, 2, 4], [3]), 'exp_name': 'main3_no

/Users/Jamin/Library/CloudStorage/GoogleDrive-jaminrader.science@gmail.com/My Drive/2024/ForcedTrend/data_methods.py:151: RuntimeWarning: Mean of empty slice
  Dmean = np.nanmean(basis, axis=(0,1,2))[None, None, None, ...]
/Users/Jamin/miniforge3/envs/forcetrend/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/Jamin/Library/CloudStorage/GoogleDrive-jaminrader.science@gmail.com/My Drive/2024/ForcedTrend/data_methods.py:174: RuntimeWarning: Mean of empty slice
  Dmean = np.nanmean(basis, axis=2)[:, :, None, ...]


*** Training model for main3_noboot
Time elapsed for training:  339.99276995658875
*** Making predictions for main3_noboot
58/58 [==============================] - 9s 151ms/step
R2 of IV: 0.903
15-yr Trends MAESS: 0.715
*** Preparing data for main3_bootI
{'data_fn': 'standard_tos_annual.npz', 'obs_fn': 'era5_sst_annual_regrid.nc', 'split_members': ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]), 'data_aug_method': 'bootstrap_Itrain2Itest', 'alter_forced_method': None, 'in_stand_method': 'self_by_member_mean_only', 'out_stand_method': 'gridpoint_stand', 'seed': 0, 'learn_rate': 0.001, 'patience': 15, 'batch_size': 64, 'max_epochs': 5, 'encoding_nodes': [100, 100], 'code_nodes': 100, 'activation': 'tanh', 'ridge': 0.0, 'variational': False, 'variational_loss': 0.0001, 'bias_init': 'random', 'kernel_init': 'random', 'skip': True, 'conv_blocks': ([['Ski

/Users/Jamin/Library/CloudStorage/GoogleDrive-jaminrader.science@gmail.com/My Drive/2024/ForcedTrend/data_methods.py:151: RuntimeWarning: Mean of empty slice
  Dmean = np.nanmean(basis, axis=(0,1,2))[None, None, None, ...]
/Users/Jamin/miniforge3/envs/forcetrend/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/Jamin/Library/CloudStorage/GoogleDrive-jaminrader.science@gmail.com/My Drive/2024/ForcedTrend/data_methods.py:174: RuntimeWarning: Mean of empty slice
  Dmean = np.nanmean(basis, axis=2)[:, :, None, ...]


*** Training model for main3_bootI
Time elapsed for training:  1184.8233180046082
*** Making predictions for main3_bootI
156/156 [==============================] - 25s 162ms/step
R2 of IV: 0.917
15-yr Trends MAESS: 0.719
*** Preparing data for main3_bootF
{'data_fn': 'standard_tos_annual.npz', 'obs_fn': 'era5_sst_annual_regrid.nc', 'split_members': ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]), 'data_aug_method': 'bootstrap_Ftrain2Ftest', 'alter_forced_method': None, 'in_stand_method': 'self_by_member_mean_only', 'out_stand_method': 'gridpoint_stand', 'seed': 0, 'learn_rate': 0.001, 'patience': 15, 'batch_size': 64, 'max_epochs': 5, 'encoding_nodes': [100, 100], 'code_nodes': 100, 'activation': 'tanh', 'ridge': 0.0, 'variational': False, 'variational_loss': 0.0001, 'bias_init': 'random', 'kernel_init': 'random', 'skip': True, 'conv_blocks': ([['Sk

...

*** Training model for main0_I


Time elapsed for training:  351.3137788772583
*** Making predictions for main0_I
156/156 [==============================] - 24s 152ms/step
R2 of IV: 0.883
15-yr Trends MAESS: 0.700
*** Preparing data for main0_F
{'data_fn': 'standard_tos_annual.npz', 'obs_fn': 'era5_sst_annual_regrid.nc', 'split_members': ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]), 'data_aug_method': 'Ftrain2Ftest', 'alter_forced_method': None, 'in_stand_method': 'self_by_member_mean_only', 'out_stand_method': 'gridpoint_stand', 'seed': 0, 'learn_rate': 0.001, 'patience': 15, 'batch_size': 64, 'max_epochs': 5, 'encoding_nodes': [100, 100], 'code_nodes': 100, 'activation': 'tanh', 'ridge': 0.0, 'variational': False, 'variational_loss': 0.0001, 'bias_init': 'random', 'kernel_init': 'random', 'skip': True, 'conv_blocks': ([['Skip'], ['Conv', 32, 3, 1, 'relu'], ['Conv', 32, 3, 1, '

/Users/Jamin/Library/CloudStorage/GoogleDrive-jaminrader.science@gmail.com/My Drive/2024/ForcedTrend/data_methods.py:151: RuntimeWarning: Mean of empty slice
  Dmean = np.nanmean(basis, axis=(0,1,2))[None, None, None, ...]
/Users/Jamin/miniforge3/envs/forcetrend/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/Jamin/Library/CloudStorage/GoogleDrive-jaminrader.science@gmail.com/My Drive/2024/ForcedTrend/data_methods.py:174: RuntimeWarning: Mean of empty slice
  Dmean = np.nanmean(basis, axis=2)[:, :, None, ...]


*** Training model for main0_F


Time elapsed for training:  351.4853639602661
*** Making predictions for main0_F
156/156 [==============================] - 25s 161ms/step
R2 of IV: 0.897
15-yr Trends MAESS: 0.665
